## Finding XMM `OBSID` for a sky source

Using the TAP interface

In [16]:
from astropy import units as u
from astropy.coordinates import SkyCoord

from astroquery.simbad import Simbad
from astroquery.utils.tap.core import TapPlus


In [48]:
nxsa = TapPlus(url="http://nxsa.esac.esa.int/tap-server/tap")
tables = nxsa.load_tables(only_names=False)
for table in (tables):
  #print(table.get_qualified_name())
  if ("public_observations" in table.get_qualified_name()):
  #if ("all_observations" in table.get_qualified_name()):
        print (table)
        break

Created TAP+ (v1.0.1) - Connection:
	Host: nxsa.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Retrieving tables...
Parsing tables...
Done.
TAP Table name: public.public.v_public_observations
Description: 
Num. columns: 29


In [49]:
for col in table.columns:
    print (f"{col.name},{col.description},{col.datatype}")

bii,Galactic Latitude [degrees],None
dec,Declination J2000 [degrees] ,None
duration,Exposure on time,None
end_utc,End date of the observation in UTC,None
footprint_fov,,None
heasarc_code,HEASARC Object Classification,None
lii,Galactic Longitude [degrees],None
moving_target,If the observation contains a moving target,None
observation_equatorial_spoint,,None
observation_fov_scircle,,None
observation_galactic_spoint,,None
observation_id,The XMM-Newton observation identification.,None
observation_oid,Internal identifier for the observation,None
odf_proc_date,,None
odf_version,,None
position_angle,Position Angle [degrees] ,None
pps_proc_date,,None
pps_version,,None
proposal_id,,None
proposal_oid,Internal proposal identifier,None
proprietary_end_date,Propietary period expiry date,None
ra,Right Ascension [degrees],None
ra_nom,Right Ascension ,None
revolution,The XMM-Newton revolution number.,None
sas_version,,None
start_utc,Start date of the observation in UTC,None
stc_s,,None
target,Target n

In [22]:
def resolve_name(name,verbose=True):
    t = Simbad.query_object(name)
    if (len(t) < 1):
        print (f"Target name {name} not found in Simbad. Please with anither name.")
        return None
    #
    if (verbose):
        print (t)
    return SkyCoord(t['RA'][0],t['DEC'][0],unit=(u.hour,u.deg))

In [23]:
# search target
target = 'Mrk1044'
c = resolve_name(target)
#ra = "12h10m32.5s"
#dec= "+39d24m20.0s"
#c = SkyCoord(ra, dec, frame='icrs')

MAIN_ID        RA           DEC      ... COO_WAVELENGTH     COO_BIBCODE    
            "h:m:s"       "d:m:s"    ...                                   
-------- ------------- ------------- ... -------------- -------------------
Mrk 1044 02 30 05.5224 -08 59 53.208 ...              O 2018yCat.1345....0G


In [21]:
c

<SkyCoord (ICRS): (ra, dec) in deg
    (37.52301, -8.99811333)>

In [78]:
#
# construct the query
#
radius = 6.0/3600.0 # 
# table to use
tap_cat = "v_public_observations"
#
circle = f"circle('ICRS',{c.ra.value:.4f},{c.dec.value:.4f},{radius:.4f})"
#dist = f"DISTANCE(POINT('ICRS',t2.ra,t2.dec),POINT('ICRS',{c.ra.value:.4f},{c.dec.value:.4f}))"
# 
# now combine two tables to include the observing mode
# instrument_mode_oid for EPN is from 60 to 69, will only search for those
#
my_query = f"SELECT t2.observation_id,t2.ra,t2.dec,t2.revolution,t2.duration,t3.mode_friendly_name,t1.filter,t2.with_science" + \
" FROM v_exposure AS t1, v_public_observations AS t2, v_instrument_mode AS t3" + \
" WHERE " + \
f"(1=INTERSECTS(t2.observation_fov_scircle,{circle})) AND " + \
"(t1.observation_oid = t2.observation_oid) AND " + \
"(t1.instrument_mode_oid = t3.instrument_mode_oid)"
#"(t1.instrument_mode_oid BETWEEN 60 and 69) and " + \
print (my_query)

SELECT t2.observation_id,t2.ra,t2.dec,t2.revolution,t2.duration,t3.mode_friendly_name,t1.filter,t2.with_science FROM v_exposure AS t1, v_public_observations AS t2, v_instrument_mode AS t3 WHERE (1=INTERSECTS(t2.observation_fov_scircle,circle('ICRS',37.5230,-8.9981,0.0017))) AND (t1.observation_oid = t2.observation_oid) AND (t1.instrument_mode_oid = t3.instrument_mode_oid)


In [79]:
#
# run the query
#
job = nxsa.launch_job(my_query)


In [72]:
job.results

observation_id,ra,dec,revolution,duration,mode_friendly_name,filter,with_science
,deg,deg,,,,,
object,float64,float64,int32,int64,object,object,bool
0112600301,37.520835,-8.998055,480,8375,Small Window,MEDIUM,True
0112600301,37.520835,-8.998055,480,8375,Spectroscopy HER + SES,7,True
0112600301,37.520835,-8.998055,480,8375,Image,V,True
0112600301,37.520835,-8.998055,480,8375,Image,V,True
0112600301,37.520835,-8.998055,480,8375,Small Window,MEDIUM,True
0112600301,37.520835,-8.998055,480,8375,Image,V,True
0112600301,37.520835,-8.998055,480,8375,Image,V,True
0112600301,37.520835,-8.998055,480,8375,Small Window,MEDIUM,True
